## 📌 Descripción del Problema

Resolver un **Sudoku clásico** consiste en llenar una cuadrícula de \(9 \times 9\) con números del 1 al 9, cumpliendo con las siguientes reglas:

- Cada fila debe contener exactamente una vez cada número del 1 al 9.
- Cada columna debe contener exactamente una vez cada número del 1 al 9.
- Cada una de las 9 subcuadrículas \(3 \times 3\) (subcuadros) también debe contener exactamente una vez cada número del 1 al 9.
- Algunas celdas tienen valores predefinidos (datos del tablero inicial).

El objetivo es encontrar una asignación de valores a las celdas que cumpla con todas las restricciones anteriores.

---

## 🧮 Modelo Matemático

### Variables

$$
y_{r,c,v} =
\begin{cases}
1 & \text{si se asigna el valor } v \text{ a la celda } (r,c) \\
0 & \text{en otro caso}
\end{cases}
$$

donde \( r, c, v \in \{1, 2, ..., 9\} \)

---

### Función Objetivo

Este es un problema de **factibilidad**, por lo tanto, la función objetivo es arbitraria (puede ser constante):

$$
\min 1
$$

---

### Restricciones

#### 1. Cada fila debe contener exactamente un valor \(v\):

$$
\sum_{c=1}^{9} y_{r,c,v} = 1 \quad \forall r \in \{1,\dots,9\},\ \forall v \in \{1,\dots,9\}
$$

#### 2. Cada columna debe contener exactamente un valor \(v\):

$$
\sum_{r=1}^{9} y_{r,c,v} = 1 \quad \forall c \in \{1,\dots,9\},\ \forall v \in \{1,\dots,9\}
$$

#### 3. Cada subcuadro \(3 \times 3\) debe contener exactamente un valor \(v\):

Para cada subcuadro \(s \in \{1,\dots,9\}\), definido por un conjunto de coordenadas \((r,c)\in S_s\):

$$
\sum_{(r,c) \in S_s} y_{r,c,v} = 1 \quad \forall v \in \{1,\dots,9\}
$$

#### 4. Cada celda debe tener exactamente un valor:

$$
\sum_{v=1}^{9} y_{r,c,v} = 1 \quad \forall r, c \in \{1,\dots,9\}
$$

#### 5. Valores dados del tablero (si los hay):

Para cada celda predefinida con valor \(v\) en la posición \((r,c)\):

$$
y_{r,c,v} = 1
$$


In [1]:
import pyomo.environ as pyo
from pyomo.core.expr.current import LinearExpression

# Definimos el mapeo de subcuadrantes a celdas
subsq_to_row_col = {
    1: [(i,j) for i in range(1,4) for j in range(1,4)],
    2: [(i,j) for i in range(1,4) for j in range(4,7)],
    3: [(i,j) for i in range(1,4) for j in range(7,10)],
    4: [(i,j) for i in range(4,7) for j in range(1,4)],
    5: [(i,j) for i in range(4,7) for j in range(4,7)],
    6: [(i,j) for i in range(4,7) for j in range(7,10)],
    7: [(i,j) for i in range(7,10) for j in range(1,4)],
    8: [(i,j) for i in range(7,10) for j in range(4,7)],
    9: [(i,j) for i in range(7,10) for j in range(7,10)]
}

def create_sudoku_model(board):
    model = pyo.ConcreteModel()

    model.ROWS = pyo.RangeSet(1,9)
    model.COLS = pyo.RangeSet(1,9)
    model.VALUES = pyo.RangeSet(1,9)
    model.SUBSQUARES = pyo.RangeSet(1,9)

    model.y = pyo.Var(model.ROWS, model.COLS, model.VALUES, within=pyo.Binary)

    # Fijar valores dados
    for (r, c, v) in board:
        model.y[r, c, v].fix(1)

    # Función objetivo trivial (problema de factibilidad)
    model.obj = pyo.Objective(expr=1.0)

    # Restricción: una única aparición de cada valor en cada fila
    def row_rule(model, r, v):
        return LinearExpression(
            constant=0,
            linear_coefs=[1]*9,
            linear_vars=[model.y[r, c, v] for c in model.COLS]
        ) == 1
    model.RowCon = pyo.Constraint(model.ROWS, model.VALUES, rule=row_rule)

    # Restricción: una única aparición de cada valor en cada columna
    def col_rule(model, c, v):
        return LinearExpression(
            constant=0,
            linear_coefs=[1]*9,
            linear_vars=[model.y[r, c, v] for r in model.ROWS]
        ) == 1
    model.ColCon = pyo.Constraint(model.COLS, model.VALUES, rule=col_rule)

    # Restricción: una única aparición de cada valor en cada subcuadro
    def subsquare_rule(model, s, v):
        coords = subsq_to_row_col[s]
        return LinearExpression(
            constant=0,
            linear_coefs=[1]*9,
            linear_vars=[model.y[r, c, v] for (r, c) in coords]
        ) == 1
    model.SquareCon = pyo.Constraint(model.SUBSQUARES, model.VALUES, rule=subsquare_rule)

    # Restricción: cada celda debe tener un único valor
    def cell_rule(model, r, c):
        return LinearExpression(
            constant=0,
            linear_coefs=[1]*9,
            linear_vars=[model.y[r, c, v] for v in model.VALUES]
        ) == 1
    model.CellCon = pyo.Constraint(model.ROWS, model.COLS, rule=cell_rule)

    return model


expression symbols from pyomo.core.expr  (deprecated in 6.6.2) (called from
C:\Users\a_cor\AppData\Local\Temp\ipykernel_12800\120685233.py:2)


In [3]:
from pyomo.opt import SolverFactory, TerminationCondition

# Tablero inicial (como en el libro)
board = [
    (1,1,5),(1,2,3),(1,5,7),
    (2,1,6),(2,4,1),(2,5,9),(2,6,5),
    (3,2,9),(3,3,8),(3,8,6),
    (4,1,8),(4,5,6),(4,9,3),
    (5,1,4),(5,4,8),(5,6,3),(5,9,1),
    (6,1,7),(6,5,2),(6,9,6),
    (7,2,6),(7,7,2),(7,8,8),
    (8,4,4),(8,5,1),(8,6,9),(8,9,5),
    (9,5,8),(9,8,7),(9,9,9)
]

model = create_sudoku_model(board)

solver = SolverFactory('glpk')
result = solver.solve(model)

# Mostrar solución
def print_solution(model):
    for r in model.ROWS:
        fila = ''
        for c in model.COLS:
            for v in model.VALUES:
                if pyo.value(model.y[r, c, v]) >= 0.5:
                    fila += f'{v} '
        print(fila)

if result.solver.termination_condition == TerminationCondition.optimal:
    print("✅ Sudoku resuelto:")
    print_solution(model)
else:
    print("❌ No se encontró solución.")


✅ Sudoku resuelto:
5 3 4 6 7 8 9 1 2 
6 7 2 1 9 5 3 4 8 
1 9 8 3 4 2 5 6 7 
8 5 9 7 6 1 4 2 3 
4 2 6 8 5 3 7 9 1 
7 1 3 9 2 4 8 5 6 
9 6 1 5 3 7 2 8 4 
2 8 7 4 1 9 6 3 5 
3 4 5 2 8 6 1 7 9 


In [4]:
# --------------------------------------------------
# 1. Importaciones
# --------------------------------------------------
import random
from pyomo.environ import *
from pyomo.opt import SolverFactory, TerminationCondition

# --------------------------------------------------
# 2. Función para generar pistas aleatorias válidas
# --------------------------------------------------
def generar_pistas_aleatorias(n_pistas):
    pistas = {}
    intentos = 0
    while len(pistas) < n_pistas and intentos < 500:
        i = random.randint(1, 9)
        j = random.randint(1, 9)
        if (i, j) in pistas:
            continue
        k = random.randint(1, 9)

        if all([
            all(k != pistas.get((i, jj), 0) for jj in range(1, 10)),
            all(k != pistas.get((ii, j), 0) for ii in range(1, 10)),
            all(
                k != pistas.get((ii, jj), 0)
                for ii in range(3*((i-1)//3)+1, 3*((i-1)//3)+4)
                for jj in range(3*((j-1)//3)+1, 3*((j-1)//3)+4)
            )
        ]):
            pistas[(i, j)] = k
        intentos += 1
    return pistas

# --------------------------------------------------
# 3. Bucle para generar una instancia resoluble
# --------------------------------------------------
solucion_encontrada = False
intentos = 0
max_intentos = 10  # Evita ciclos infinitos

while not solucion_encontrada and intentos < max_intentos:
    print(f"\nIntento #{intentos + 1}")
    pistas = generar_pistas_aleatorias(25)  # Puedes cambiar la cantidad
    model = ConcreteModel()

    # Índices
    model.I = Set(initialize=range(1, 10))
    model.J = Set(initialize=range(1, 10))
    model.K = Set(initialize=range(1, 10))

    # Variables binarias
    model.x = Var(model.I, model.J, model.K, domain=Binary)

    # Restricciones
    def cell_unique_rule(model, i, j):
        return sum(model.x[i, j, k] for k in model.K) == 1
    model.cell_unique = Constraint(model.I, model.J, rule=cell_unique_rule)

    def row_unique_rule(model, i, k):
        return sum(model.x[i, j, k] for j in model.J) == 1
    model.row_unique = Constraint(model.I, model.K, rule=row_unique_rule)

    def col_unique_rule(model, j, k):
        return sum(model.x[i, j, k] for i in model.I) == 1
    model.col_unique = Constraint(model.J, model.K, rule=col_unique_rule)

    def block_unique_rule(model, br, bc, k):
        return sum(
            model.x[i, j, k]
            for i in range(3*br+1, 3*br+4)
            for j in range(3*bc+1, 3*bc+4)
        ) == 1
    model.block_unique = Constraint(
        range(3), range(3), model.K, rule=block_unique_rule
    )

    # Pistas fijas
    model.clues = ConstraintList()
    for (i, j), val in pistas.items():
        model.clues.add(model.x[i, j, val] == 1)

    # Objetivo: minimizar movimientos (nuevas celdas)
    model.obj = Objective(
        expr=sum(
            model.x[i, j, k]
            for i in model.I for j in model.J for k in model.K
            if (i, j) not in pistas or pistas[(i, j)] != k
        ),
        sense=minimize
    )

    # Resolver
    solver = SolverFactory('glpk')
    results = solver.solve(model, tee=False)

    if results.solver.termination_condition == TerminationCondition.optimal:
        solucion_encontrada = True
        print("¡Sudoku resuelto correctamente!\n")
        print("Sudoku completado:")
        for i in model.I:
            fila = []
            for j in model.J:
                for k in model.K:
                    if value(model.x[i, j, k]) > 0.5:
                        fila.append(str(k))
                        break
            print(" ".join(fila))
    else:
        print("Sudoku sin solución, generando otro...")
        intentos += 1

if not solucion_encontrada:
    print("\nNo se logró generar un Sudoku resoluble tras varios intentos.")



Intento #1
Sudoku sin solución, generando otro...

Intento #2
Sudoku sin solución, generando otro...

Intento #3
Sudoku sin solución, generando otro...

Intento #4
Sudoku sin solución, generando otro...

Intento #5
Sudoku sin solución, generando otro...

Intento #6
Sudoku sin solución, generando otro...

Intento #7
¡Sudoku resuelto correctamente!

Sudoku completado:
7 3 5 8 4 6 9 1 2
9 6 2 7 3 1 4 5 8
4 8 1 5 2 9 7 3 6
2 5 6 4 1 7 3 8 9
1 4 3 2 9 8 5 6 7
8 7 9 6 5 3 2 4 1
6 9 4 3 8 2 1 7 5
5 1 8 9 7 4 6 2 3
3 2 7 1 6 5 8 9 4
